In [1]:
from hypex.experiments.homogeneity import HOMOGENEITY_TEST
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
test = HOMOGENEITY_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [4]:
result.analysis_tables

{'GroupSizes┴┴':                       1
 control size    4936.00
 control size %    49.36
 test size       5064.00
 test size %       50.64,
 'GroupDifference┴┴':                                    1
 post_spends control mean  420.046619
 post_spends difference     63.424045
 post_spends difference %   15.099287
 post_spends test mean     483.470664,
 'TTest┴┴':          p-value   statistic  pass
 TTest┴┴      0.0 -135.560001  True,
 'KSTest┴┴':           p-value  statistic  pass
 KSTest┴┴      0.0     0.8959  True,
 'OneAAStatAnalyzer┴┴':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0            0.0         1.0             0.0          1.0              0.0}

In [5]:
from hypex.reporters.homo import HomoReporter

HomoReporter().report(result)

{'1 post_spends control mean': 420.04661894471457,
 '1 post_spends difference': 63.42404544004944,
 '1 post_spends difference %': 15.099287217068902,
 '1 post_spends test mean': 483.470664384764,
 '1 control size': 4936.0,
 '1 control size %': 49.36,
 '1 test size': 5064.0,
 '1 test size %': 50.63999999999999,
 'TTest p-value': 0.0,
 'TTest pass': 1.0,
 'KSTest p-value': 0.0,
 'KSTest pass': 1.0,
 'mean test score': 0.0}